In [1]:
# THIS MUST BE THE FIRST CELL IN THE NOTEBOOK
%load_ext autoreload
%autoreload 2
import time
import os, sys
DEVEL_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel")
if DEVEL_SRC not in sys.path: 
    sys.path.append(DEVEL_SRC)

PRE2POST_SRC = os.path.join(DEVEL_SRC, "use-cases", "Pre2post_bdp") # TODO when - is removed, remove also this line and adapt imports
if PRE2POST_SRC not in sys.path: 
    sys.path.append(PRE2POST_SRC)

import pykhaos.utils.pyspark_configuration as pyspark_config

start_time = time.time()
sc, spark, sql_context = pyspark_config.get_spark_session(app_name="Manage predictions", log_level="OFF")
print("Ended spark session: {} secs | default parallelism={}".format(time.time()-start_time, sc.defaultParallelism))

from pyspark.sql.functions import (udf,col,max as sql_max, when, isnull, concat, lpad, trim, lit, sum as sql_sum, length, upper)


Ended spark session: 35.3235361576 secs | default parallelism=9


In [39]:
import pandas as pd

def assign_decile(df_pred_risk):
    df_pred_risk.sort_values(by="model_score", ascending=False, inplace=True)
    df_pred_risk["decile"]=pd.qcut(df_pred_risk["model_score"],10,retbins=False,labels=False)
    df_pred_risk["decile"] += 1 

In [55]:
import matplotlib.pyplot as plt
%matplotlib inline 

SHOW_PLOT = False
FILE_EXTENSION = "txt"
SEP = "|"

PRED_NAMES_LIST = {"preds_onlymob_20180802_1210"         : [0.20, "preds_amdocs_onlymob_201807"],  # coger 20% (aprox 140k)
                   "preds_allmob_masmovil_20180802_2025" : [0.20, "preds_masmovil_amdocs_201807_processed"],
                   "preds_convergent_20180802_1820"      : [0.20, "preds_amdocs_conv_201807"] # coger 20% (aprox 140k)
                  } 

for PRED_NAME, data_rows in PRED_NAMES_LIST.items():
    
    num_rows = data_rows[0]
    filename_pred = data_rows[1]
    print(PRED_NAME)
    start_time = time.time()
    latest_predictions = (spark.read.table("tests_es.jvmm_amdocs_churn_scores")
                .where(col("pred_name") == PRED_NAME))
    
    df_pred = latest_predictions.toPandas()
    df_pred = df_pred[["campo2","model_score"]]
    df_pred.rename(columns={"campo2" : "msisdn"}, inplace=True)
    
    filename_csv = "/var/SP/data/home/csanc109/data/delivery/amdocs_churn/{}.{}".format(filename_pred, FILE_EXTENSION)
    filename_risk = filename_csv.replace("."+FILE_EXTENSION,"")+"_risk."+FILE_EXTENSION

    df_pred.sort_values(by="model_score", ascending=False, inplace=True)
    df_pred.reset_index(drop=True,inplace=True)
    
    df_pred_risk = df_pred.head(int(len(df_pred)*0.20)).copy()
    assign_decile(df_pred_risk)
    
       
    assign_decile(df_pred)
    df_pred.rename(columns={"decile" : "decile_all"}, inplace=True)
    
    df_pred_risk.rename(columns={"decile" : "decile_risk"}, inplace=True)

    # decile_risk: 0 si no esta en risk
    df_pred = pd.merge(df_pred, df_pred_risk[["msisdn", "decile_risk"]], on="msisdn", how="left")
    df_pred["decile_risk"].fillna(0,inplace=True)

    df_pred["decile_risk"] = df_pred["decile_risk"].astype(int)
    df_pred["decile_all"]  = df_pred["decile_all"].astype(int)

    df_pred[["msisdn","model_score", "decile_all", "decile_risk"]].to_csv(filename_csv, header=True, index=False, sep=SEP)
    print(filename_csv)    
       
    if "masmovil" in PRED_NAME:
        df_pred_risk.to_csv(filename_risk, header=True, index=False)
        print(filename_risk)
    else:
        df_pred_risk.rename(columns={"decile_risk" : "decile"}, inplace=True)
        df_pred_risk.to_csv(filename_risk, header=True, index=False, sep=SEP)
        print(filename_risk)
    
    if show_plot:
        n, bins, patches = plt.hist(df_pred["model_score"], bins=50)
        plt.xlabel('Model Score')
        plt.ylabel('Number of msisdns')
        plt.title(PRED_NAME)
        plt.grid(True)
        plt.show()
    
    print(len(df_pred), len(df_pred_risk)) 
    

preds_allmob_masmovil_20180802_2025
/var/SP/data/home/csanc109/data/delivery/amdocs_churn/preds_masmovil_amdocs_201807_processed.txt
/var/SP/data/home/csanc109/data/delivery/amdocs_churn/preds_masmovil_amdocs_201807_processed_risk.txt
(3036551, 607310)
preds_onlymob_20180802_1210
/var/SP/data/home/csanc109/data/delivery/amdocs_churn/preds_amdocs_onlymob_201807.txt
/var/SP/data/home/csanc109/data/delivery/amdocs_churn/preds_amdocs_onlymob_201807_risk.txt
(891926, 178385)
preds_convergent_20180802_1820
/var/SP/data/home/csanc109/data/delivery/amdocs_churn/preds_amdocs_conv_201807.txt
/var/SP/data/home/csanc109/data/delivery/amdocs_churn/preds_amdocs_conv_201807_risk.txt
(1998445, 399689)


In [56]:
df_pred.groupby(["decile_all","decile_risk"]).agg(

        {
        'model_score': [
            ('max', lambda gg: gg.max()),
            ('min', lambda gg: gg.min()),
            ('count', lambda gg: gg.count())
                 ]
        }
    )

model_score                    
                               max       min     count
decile_all decile_risk                                
1          0              0.141574  0.010113  199846.0
2          0              0.355784  0.141574  199843.0
3          0              0.404464  0.355784  199845.0
4          0              0.435554  0.404464  199844.0
5          0              0.464563  0.435554  199845.0
6          0              0.498540  0.464563  199844.0
7          0              0.546011  0.498540  199844.0
8          0              0.596558  0.546011  199845.0
9          1              0.606680  0.596558   39969.0
           2              0.618556  0.606680   39969.0
           3              0.635827  0.618556   39969.0
           4              0.682788  0.635828   39969.0
           5              0.722506  0.682788   39968.0
10         5              0.722507  0.722507       1.0
           6              0.746564  0.722507   39968.0
           7              0.764311  0.746565   39969.0
           8              0.779903  0.764311   39969.0
           9              0.795845  0.779904   39969.0
           10             0.851123  0.795845   39969.0